# Web

## Запросы и парсинг HTML

In [1]:
from urllib.request import urlopen

with urlopen('https://www.google.by') as connection:
    data = connection.read()

    real_url = connection.geturl()
    connection_info = connection.info()
    http_code = connection.getcode()

    print(http_code)

200


### Requests and Beautiful Soup

In [2]:
import requests  # для запросов

import urllib.parse  # для encode/decode url

from bs4 import BeautifulSoup  # для парсинга HTML

from IPython.core.display import display, HTML  # IPython-отображение HTMl страниц


def google(query):
    template = 'https://www.google.by/search?{}'
    url = template.format(urllib.parse.urlencode({'q' : query}))
    
    # объект Request, содержащий информацию о запросе:
    #   успешность выполнения, код, заголовки
    req = requests.get(url, timeout=3.0)  # не стоит забывать про timeout
    assert req.status_code == 200, 'request failed'
    
    soup = BeautifulSoup(req.text, "html5lib")
    
    for i, li in enumerate(soup.findAll('h3', attrs={'class': 'r'})):
        link = li.find('a')
        if link:
            link['href'] = 'http://www.google.by' + link['href']
            link = '<b>{}.</b>  {}'.format(i + 1, link)
            display(HTML(str(link)))

google('Yandex')

В IPython можно также отображать картинки

In [ ]:
from IPython.display import Image
Image('http://www.openbookproject.net/tutorials/getdown/css/images/lesson4/HTMLDOMTree.png')

### API с использованием get запросов

In [3]:
import pprint 
import requests
import json


DICTIONARY_API = 'https://dictionary.yandex.net/api/v1/dicservice.json/lookup'
YNDX_TMP_KEY = 'dict.1.1.20171114T131015Z.59d0570f825aeb6d.aa00fdbd5ee643a6cb790c0963c0946115ceef55'

query = 'API'
params = {'key' : YNDX_TMP_KEY, 'lang' : 'en-ru', 'text' : query}
    
req = requests.get(DICTIONARY_API, params=params)
pprint.pprint(json.loads(req.text)['def'])

[{'pos': 'noun',
  'text': 'API',
  'tr': [{'mean': [{'text': 'application programming interface'}],
          'pos': 'noun',
          'syn': [{'pos': 'noun', 'text': 'интерфейс API'},
                  {'pos': 'noun',
                   'text': 'интерфейс прикладного программирования'},
                  {'pos': 'noun', 'text': 'программный интерфейс приложения'}],
          'text': 'прикладной программный интерфейс'}],
  'ts': 'ˈæpɪ'}]


### API с использованием post запросов

In [4]:
import requests
import json

def get_doublegis(source, dest):
    url = 'https://catalog.api.2gis.ru/carrouting/4.0.0/moscow?key=rutnpt3272'
    payload = { "locale": "en", "point_a_name": "Start", "point_b_name": "Finish", "points": [{
                "type": "pedo",
                "x": source[1],
                "y": source[0]
            }, {
                "type": "pedo",
                "x": dest[1],
                "y": dest[0]
            }
        ], "type": "online", "region_id": "32"
    }

    data = requests.post(url, data=json.dumps(payload))
    route = json.loads(data.text)['result'][0]
    length = route['total_distance']
    eta = route['total_duration']

    return length, eta / 60

In [5]:
source, dest = (55.734242, 37.587424), (55.798485, 37.39912)
print(get_doublegis(source, dest))

(22902, 51.56666666666667)


### LXML и Requests

In [ ]:
import requests
from lxml import etree
from io import StringIO, BytesIO

data = requests.get('http://bash.im').text

parser = etree.HTMLParser()
doc = etree.parse(StringIO(data), parser)

# поиск с помощью XPATH
article_list = doc.findall("//div[@class='quote']")  # doc.iter('div')

for ell in article_list:
    content = ell.find("div[@class='text']")
    
    print(content.tag)
    print(content.attrib)
    print(content.text)
    
    # print(etree.tostring(content))

    break

## Сокеты

In [ ]:
import socket

# Create a TCP/IP socket
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

In [ ]:
server_address = ('localhost', 56789)
sock.bind(server_address)

In [ ]:
sock.listen(1)

In [ ]:
def echo(sock):
    # tells we are in server mode
    sock.listen(1)

    while True:
        # Wait for a connection
        connection, client_address = sock.accept()
        try:
            print('connection from', client_address)

            # Receive the data in small chunks and retransmit it
            while True:
                data = connection.recv(16)
                print('received "%s"' % data)
                if data:
                    print('sending data back to the client')
                    connection.sendall(data)
                else:
                    print('no more data from', client_address)
                    break

        finally:
            # Clean up the connection
            connection.close()

In [ ]:
# server example
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_address = ('localhost', 56789)
sock.bind(server_address)
echo(sock)

In [ ]:
# client example
import socket

# Create a TCP/IP socket
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Connect the socket to the port where the server is listening
server_address = ('localhost', 56789)
print('connecting to %s port %s' % server_address)
sock.connect(server_address)

try:
    # Send data
    message = 'This is a message. It will be repeated.'
    print('sending "%s"' % message)
    sock.sendall(message.encode('utf-8'))

    # Look for the response
    amount_received = 0
    amount_expected = len(message)
    
    while amount_received < amount_expected:
        data = sock.recv(16)
        amount_received += len(data)
        print('received "%s"' % data)

finally:
    print('closing socket')
    sock.close()

## Web frameworks

### Flask

Очень простое приложение

In [ ]:
import json 

from flask import Flask
from flask import Response, request

app = Flask(__name__)


@app.route('/hello',methods=['GET'])
def hello(): 
    return json.dumps({'msg' : 'hello my friend'}), 200


app.run(host='localhost', port=5000, use_reloader=False, threaded=True)

Приложение поинтереснее

In [ ]:
import json 

from flask import Flask
from flask import Response, request


app = Flask(__name__)


@app.route('/hello', methods=['GET'])
def hello(): 
    return json.dumps({'msg': 'hello my friend'}), 200


@app.route('/privet/<name>', methods=['GET'])
def hello2(name): 
    return json.dumps({'msg': 'hello my friend, {}'.format(name)}), 200


def _parse_bool(val): 
    return val.lower() == 'true'

    
@app.route('/hi/<name>/<value>', methods=['GET'])
def hi_common(name, value): 
    should_answer = request.args.get('need_answer', 'false')
    should_answer = _parse_bool(should_answer)

    if should_answer: 
        return 'hi my friend!', 200
    else: 
        return 'no answer :(', 404


@app.route('/hi/tanya/<value>', methods=['GET'])
def hi(value): 
    return 'love you!', 200


app.run(host='localhost', port=10000, use_reloader=False, threaded=True)

### Jinja

Удобная генерация html из шаблонов

In [ ]:
from jinja2 import Template
t = Template("Hello {{ something }}!")
t.render(something="World")

In [ ]:
t = Template("My favorite numbers: {% for n in range(1,10) %}<b>{{n}}</b> ""{% endfor %}")
t.render()

### Использование Jinja и Flask

In [ ]:
from flask import Flask, render_template
app = Flask(__name__)


@app.route("/")
def template_test():
    return render_template('template.html', my_string="Wheeeee!", my_list=[0, 1, 2, 3, 4, 5])


if __name__ == '__main__':
    app.run(debug=True)

Сохраните шаблон ниже в требуемый файл

```
<!DOCTYPE html>
<html>
  <head>
    <title>Flask Template Example</title>
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <link href="http://netdna.bootstrapcdn.com/bootstrap/3.0.0/css/bootstrap.min.css" rel="stylesheet" media="screen">
    <style type="text/css">
      .container {
        max-width: 500px;
        padding-top: 100px;
      }
    </style>
  </head>
  <body>
    <div class="container">
      <p>My string: {{my_string}}</p>
      <p>Value from the list: {{my_list[3]}}</p>
      <p>Loop through the list:</p>
      <ul>
        {% for n in my_list %}
        <li>{{n}}</li>
        {% endfor %}
      </ul>
    </div>
    <script src="http://code.jquery.com/jquery-1.10.2.min.js"></script>
    <script src="http://netdna.bootstrapcdn.com/bootstrap/3.0.0/js/bootstrap.min.js"></script>
  </body>
</html>
```